# Import data
Imports data as numpy array (nr_of_sequences, 16, 128, 128, 3)

In [1]:
from load_dataset_numpy import load_dataset_numpy

sequence_limit = 15
X_train, Y_train, X_train_flow_paths, X_train_warped_paths, X_train_diff_paths, X_test, Y_test, X_test_flow_paths, X_test_warped_paths, X_test_diff_paths, image_count = load_dataset_numpy(difficulty="All", sequence_limit = sequence_limit + 1)

Using TensorFlow backend.


processing folder 0, number of images in that folder 16 
total cumulative number of image sequences:  1
processing folder 1, number of images in that folder 299 
total cumulative number of image sequences:  285
processing folder 2, number of images in that folder 44 
total cumulative number of image sequences:  314
processing folder 3, number of images in that folder 50 
total cumulative number of image sequences:  349


In [ ]:
print('output shape'.format(X_train.shape))
print('number of sequences'.format(X_train.shape[0]))

# Layer structure

<img src="images/layer_structure.png">

In [13]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Conv2D, Dense, LSTM,MaxPooling2D , Flatten, TimeDistributed,Activation

im_size = 128
time_steps = 16 # len of image sequence
channels = 3
#input_shape=(5, 128, 128, 3)))

model = Sequential()
model.add(TimeDistributed(Conv2D(filters=96, kernel_size=7,  strides=2, padding='valid'), input_shape=(time_steps,im_size,im_size,channels))) # first input shape is the len of seq
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(filters=384, kernel_size=3,  strides=2, padding='valid')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Conv2D(filters=512, kernel_size=3, padding='same')))
model.add(TimeDistributed(Conv2D(filters=512, kernel_size=3, padding='same')))
model.add(TimeDistributed(Conv2D(filters=384, kernel_size=3, padding='same')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten())) #The Flatten layer is only needed because LSTM shape should have one dimension per input.
model.add(TimeDistributed(Dense(4096)))
#model.add(Reshape((x,y)))
model.add(LSTM(256, return_sequences=False)) #pole kindel return_sequence's, kui True, siis multiple outputs
#When return_sequences=True, the output shape is (batch, timeSteps, outputFeatures)
#When return_sequences=False, the output shape is (batch, outputFeatures)
model.add(TimeDistributed(Dense(2)))
model.add((Activation('softmax')))
print(model.summary())

AssertionError: 

##  TODO Fit a model
Useful reading: https://medium.com/smileinnovation/training-neural-network-with-image-sequence-an-example-with-video-as-input-c3407f7a0b0f,

https://medium.com/smileinnovation/how-to-work-with-time-distributed-data-in-a-neural-network-b8b39aa4ce00

For train data, we need to probably reformat them:

https://keras.io/api/preprocessing/timeseries/

In [2]:
X_train.shape, type(X_train)

((349, 16, 128, 128, 3), numpy.ndarray)

In [14]:
print('number of sequences', len(X_train))
print('images in one sequence ', len(X_train[0]))
print('one image length', len(X_train[0][0]))
import numpy as np
import random
#X_new = np.random.random((90, 5, 128, 128, 3)) # training data
#Y_new = np.random.random((90, 5, 128, 128, 3)) # labels
#X_new = np.array(X_train)
#print(X_new.shape)
#Y_new = Y_train
#print(len(X_new), len(X_new[0]), len(X_new[0][0]),len(X_new[0][0][0]),len(X_new[0][0][0][0]))
# 471, 16, 128, 128, 3

# TODO: right now using random parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#model.fit(X_train, Y_train, epochs=3, batch_size=64)
model.fit(X_train, Y_train, epochs=3, batch_size=64)
# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

number of sequences 349
images in one sequence  16
one image length 128


AttributeError: 'Sequential' object has no attribute '_feed_output_names'